In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import os
import numpy as np
from sklearn.metrics import average_precision_score

In [2]:
def cal_iou(predict, ground_truth):
    """
        predict: [x1, y1, x2, y2]
        ground_truth: [x1, y1, x2, y2]

        return: iou
    """
    predict_area = (predict[2] - predict[0]) * (predict[3] - predict[1])
    ground_truth_area = (ground_truth[2] - ground_truth[0]) * (ground_truth[3] - ground_truth[1])

    # get the each coordinate of intersection rectangle
    left = max(predict[0], ground_truth[0])
    right = min(predict[2], ground_truth[2])
    top = max(predict[1], ground_truth[1])
    bottom = min(predict[3], ground_truth[3])

    if left >= right or top >= bottom:
        return 0.0

    intersection_area = (right - left) * (bottom - top)
    union_area = predict_area + ground_truth_area - intersection_area
    return intersection_area / union_area

def xywh_to_xyxy(yolo_box):
    """
        yolo_box: [x, y, w, h]

        return: [x1, y1, x2, y2]
    """
    x1 = yolo_box[0] - yolo_box[2] / 2
    y1 = yolo_box[1] - yolo_box[3] / 2
    x2 = yolo_box[0] + yolo_box[2] / 2
    y2 = yolo_box[1] + yolo_box[3] / 2
    return [x1, y1, x2, y2]

def read_labels(label_paths):
    """
        label_path: list of label path

        return: list of symbol_gt
        
            0: Air Rt Unmasked
            1: Air Lt Unmasked
            2: Air Rt Masked
            3: Air Lt masked
            4: Bone Rt Unmasked
            5: Bone Lt Unmasked
            6: Bone Rt Masked
            7: Bone Lt Masked

    """
    air_rt_unmasked_gt = [] # 0
    air_lt_unmasked_gt = [] # 1
    air_rt_masked_gt = [] # 2
    air_lt_masked_gt = [] # 3
    bone_rt_unmasked_gt = [] # 4
    bone_lt_unmasked_gt = [] # 5
    bone_rt_masked_gt = [] # 6
    bone_lt_masked_gt = [] # 7
    
    for label_path in label_paths:
        with open(label_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                line = line.split()
                xywh = [float(line[1]), float(line[2]), float(line[3]), float(line[4])]
                if line[0] == '0':
                    air_rt_unmasked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '1':
                    air_lt_unmasked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '2':
                    air_rt_masked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '3':
                    air_lt_masked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '4':
                    bone_rt_unmasked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '5':
                    bone_lt_unmasked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '6':
                    bone_rt_masked_gt.append(xywh_to_xyxy(xywh))
                elif line[0] == '7':
                    bone_lt_masked_gt.append(xywh_to_xyxy(xywh))
    
    return air_rt_unmasked_gt, air_lt_unmasked_gt, air_rt_masked_gt, air_lt_masked_gt, bone_rt_unmasked_gt, bone_lt_unmasked_gt, bone_rt_masked_gt, bone_lt_masked_gt

def get_xyxyn(results):
    """
        results: list of result

        return: graph_pred, table_pred

            0: Air Rt Unmasked
            1: Air Lt Unmasked
            2: Air Rt Masked
            3: Air Lt masked
            4: Bone Rt Unmasked
            5: Bone Lt Unmasked
            6: Bone Rt Masked
            7: Bone Lt Masked
    """
    air_rt_unmasked_pred = [] # 0
    air_lt_unmasked_pred = [] # 1
    air_rt_masked_pred = [] # 2
    air_lt_masked_pred = [] # 3
    bone_rt_unmasked_pred = [] # 4
    bone_lt_unmasked_pred = [] # 5
    bone_rt_masked_pred = [] # 6
    bone_lt_masked_pred = [] # 7
    
    for result in results:
        cls = result.boxes.cls.cpu().numpy()
        xywh = result.boxes.xywhn.cpu().numpy()
        for i, c in enumerate(cls):
            bbox = xywh_to_xyxy(xywh[i])
            if c == 0:
                air_rt_unmasked_pred.append(bbox)
            elif c == 1:
                air_lt_unmasked_pred.append(bbox)
            elif c == 2:
                air_rt_masked_pred.append(bbox)
            elif c == 3:
                air_lt_masked_pred.append(bbox)
            elif c == 4:
                bone_rt_unmasked_pred.append(bbox)
            elif c == 5:
                bone_lt_unmasked_pred.append(bbox)
            elif c == 6:
                bone_rt_masked_pred.append(bbox)
            elif c == 7:
                bone_lt_masked_pred.append(bbox)
    
    return air_rt_unmasked_pred, air_lt_unmasked_pred, air_rt_masked_pred, air_lt_masked_pred, bone_rt_unmasked_pred, bone_lt_unmasked_pred, bone_rt_masked_pred, bone_lt_masked_pred

def get_mean_iou(ground_truth, predict):
    """
        ground_truth: list of ground truth
        predict: list of predict

        return: mean_iou
    """
    total_iou = 0
    if len(ground_truth) != len(predict):
        predict = predict + [[0,0,0,0] for i in range(len(ground_truth) - len(predict))]

    for i in range(len(ground_truth)):
        iou = cal_iou(ground_truth[i], predict[i])
        total_iou += iou

    mean_iou = total_iou / len(ground_truth)
    return mean_iou

In [3]:
model = YOLO('runs\\detect\\yolov5\\weights\\best.pt')
model.val()
# image path
test_image_dir = Path('data/images/test')
test_image_path = [str(file) for file in test_image_dir.glob('*')]
train_image_dir = Path('data/images/train')
train_image_path = [str(file) for file in train_image_dir.glob('*')]
# label path
test_label_dir = Path('data/labels/test')
test_label_path = [str(file) for file in test_label_dir.glob('*')]
train_label_dir = Path('data/labels/train')
train_label_path = [str(file) for file in train_label_dir.glob('*')]

Ultralytics YOLOv8.0.118  Python-3.10.6 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLOv5n summary (fused): 193 layers, 2504504 parameters, 0 gradients
val: Scanning C:\Users\earth\OneDrive\Desktop\Audiogram\src\data_extracter\symbol_detection\data\labels\test.cache... 40 images, 0 backgrounds, 0 corrupt: 100%|██████████| 40/40 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:05<00:00,  1.80s/it]
                   all         40        956      0.917      0.867      0.886      0.411
       Air Rt Unmasked         40        288      0.957      0.937      0.957      0.496
       Air Lt Unmasked         40        325      0.961      0.905      0.925      0.417
         Air Rt Masked         40         37          1      0.763      0.833      0.367
         Air Lt masked         40         10       0.98        0.9      0.986      0.523
      Bone Rt Unmasked         40        10

In [4]:
# build the df that summary xyxn from results and labels in each class
results_df = pd.DataFrame(columns=['image_path', 'air_rt_unmasked', 'air_lt_unmasked', 'air_rt_masked', 'air_lt_masked', 'bone_rt_unmasked', 'bone_lt_unmasked', 'bone_rt_masked', 'bone_lt_masked']) 
labels_df = pd.DataFrame(columns=['image_path', 'air_rt_unmasked', 'air_lt_unmasked', 'air_rt_masked', 'air_lt_masked', 'bone_rt_unmasked', 'bone_lt_unmasked', 'bone_rt_masked', 'bone_lt_masked'])
image_path = test_image_path
label_path = test_label_path
for i in range(len(image_path)):
    results = model(image_path[i])
    air_rt_unmasked_pred, air_lt_unmasked_pred, air_rt_masked_pred, air_lt_masked_pred, bone_rt_unmasked_pred, bone_lt_unmasked_pred, bone_rt_masked_pred, bone_lt_masked_pred = get_xyxyn(results)
    air_rt_unmasked_gt, air_lt_unmasked_gt, air_rt_masked_gt, air_lt_masked_gt, bone_rt_unmasked_gt, bone_lt_unmasked_gt, bone_rt_masked_gt, bone_lt_masked_gt = read_labels([label_path[i]])
    results_df.loc[i] = [image_path[i], air_rt_unmasked_pred, air_lt_unmasked_pred, air_rt_masked_pred, air_lt_masked_pred, bone_rt_unmasked_pred, bone_lt_unmasked_pred, bone_rt_masked_pred, bone_lt_masked_pred]
    labels_df.loc[i] = [image_path[i], air_rt_unmasked_gt, air_lt_unmasked_gt, air_rt_masked_gt, air_lt_masked_gt, bone_rt_unmasked_gt, bone_lt_unmasked_gt, bone_rt_masked_gt, bone_lt_masked_gt]


image 1/1 c:\Users\earth\OneDrive\Desktop\autoaudiogram\src\data_extracter\symbol_detection\data\images\test\107.jpg: 640x640 8 Air Rt Unmaskeds, 8 Air Lt Unmaskeds, 4 Bone Rt Unmaskeds, 4 Bone Lt Unmaskeds, 54.3ms
Speed: 6.0ms preprocess, 54.3ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\earth\OneDrive\Desktop\autoaudiogram\src\data_extracter\symbol_detection\data\images\test\112.jpg: 640x640 8 Air Rt Unmaskeds, 8 Air Lt Unmaskeds, 2 Bone Rt Unmaskeds, 6 Bone Lt Unmaskeds, 64.1ms
Speed: 7.0ms preprocess, 64.1ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\earth\OneDrive\Desktop\autoaudiogram\src\data_extracter\symbol_detection\data\images\test\113.jpg: 640x640 9 Air Rt Unmaskeds, 8 Air Lt Unmaskeds, 4 Bone Rt Unmaskeds, 2 Bone Lt Unmaskeds, 3 Bone Lt Maskeds, 43.5ms
Speed: 5.0ms preprocess, 43.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 c:\Users\earth\OneDrive\Desktop

In [6]:
# calculate iou socre for each class
iou_scores = {
    'air_rt_um': [],
    'air_lt_um': [],
    'air_rt_m': [],
    'air_lt_m': [],
    'bone_rt_um': [],
    'bone_lt_um': [],
    'bone_rt_m': [],
    'bone_lt_m': []
}

# Define a list of class names for easy access
class_names = list(iou_scores.keys())

for i in range(len(results_df)):
    for j in range(8):
        if len(results_df.iloc[i, j+1]) != 0 and len(labels_df.iloc[i, j+1]) != 0:
            predict = sorted(results_df.iloc[i][j+1])
            ground_truth = sorted(labels_df.iloc[i][j+1])
            iou = get_mean_iou(ground_truth, predict)
            # Append the IoU score to the appropriate class
            iou_scores[class_names[j]].append(iou)

# Calculate the mean IoU score for all classes
mean_iou_scores = []
for iou_list in iou_scores.values():
    mean_iou_scores.append(np.mean(iou_list))
# print(mean_iou_scores)
print(np.mean(mean_iou_scores)) 

0.6194493865395182
